In [1]:
import os
from _paths import nomeroff_net_dir
from nomeroff_net.tools.datasets_tools import check_ocr_model


In [9]:
check_ocr_model(os.path.join(nomeroff_net_dir,  './data/dataset/TextDetector/ocr_example/val'))

LOADED 1 ANNOTATIONS
LOADED 1 IMAGES
PREDICTED 1 IMAGES
Predicted 'Y929', real: '0038SC' in file /mnt/data/var/www/nomeroff-net/tools/py/../.././data/dataset/TextDetector/ocr_example/val/ann/0038SC-0.json
Error detection count: 1
Accuracy: 0.0
